In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama

# === 1. Setup ===
def load_and_prepare(pdf_path):
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = splitter.split_documents(docs)
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = Chroma.from_documents(splits, embedding=embeddings)
    return vectorstore.as_retriever(search_kwargs={"k": 3})

def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

# === 2. Prompt Variants ===
basic_prompt = PromptTemplate.from_template("""
You are a helpful assistant for medical affairs.

Use the following context to answer the question concisely and accurately.

Context:
{context}

Question:
{question}

Answer:
""")

cot_prompt = PromptTemplate.from_template("""
You are a biomedical research assistant.

Use the following context to answer the question, reasoning step by step.

Context:
{context}

Question:
{question}

Let's think through this:
""")

few_shot_prompt = PromptTemplate.from_template("""
You are a biomedical assistant. Here is how you should respond.

Example:

Question: What is the role of real-world data in clinical studies?

Answer:
Real-world data helps identify population-level patterns outside of clinical trials. It supports regulatory decisions and comparative effectiveness studies.

Now answer this:

Question:
{question}

Answer:
""")

# === 3. Build chain ===
def build_chain(prompt_template, retriever):
    llm = ChatOllama(model="llama3")
    return (
        {
            "context": retriever | format_docs,
            "question": RunnablePassthrough()
        }
        | prompt_template
        | llm
        | StrOutputParser()
    )

# === 4. Main Interactive App ===
def main():
    retriever = load_and_prepare("/Users/kryptonempyrean/Desktop/TS3043166.pdf")

    options = {
        "1": ("Basic Prompt", basic_prompt),
        "2": ("Chain-of-Thought Prompt", cot_prompt),
        "3": ("Few-Shot Prompt", few_shot_prompt)
    }

    print("🧠 Choose prompt style:")
    for key, (name, _) in options.items():
        print(f"{key}. {name}")

    choice = input("Your choice (1/2/3): ").strip()
    if choice not in options:
        print("Invalid choice.")
        return

    prompt_name, prompt_template = options[choice]
    print(f"✅ Using: {prompt_name}")

    chain = build_chain(prompt_template, retriever)

    while True:
        query = input("🧠 You: ")
        if query.lower() in ["exit", "quit"]:
            break
        response = chain.invoke(query)
        print("\n🤖 Bot:", response, "\n")

if __name__ == "__main__":
    main()
